In [1]:
import numpy as np
import pandas as pd
import math
from scipy.optimize import fsolve

# 測試的

In [2]:
def f(x):
    return x + 1000
 
def g(y):
    return y * 3

def bestweight_pre(w):
    w0 = w[0]
    w1 = w[1]
    return [
        f(w0) + 3*g(w0) - 1,
        f(w1) + 3*g(w1) - 1
    ]

r = fsolve(bestweight_pre, [0,0])
print(r)

[-99.9 -99.9]


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


# 開發中的

In [3]:
def f1(x,arr_returns):
    arr_returns - rfrate 
    return sum( np.exp(-1*arr_returns/x) ) - len(arr_returns)
def get_riskiness_r(arr_returns):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,arr_returns)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]

In [4]:
def risk(x):
    guess  = 10**(-5)
    while (guess<1000):
        risk2 = fsolve(f1,guess,x)  #引用函數f1
        if risk2[0] != guess:
            break
        guess = guess*10
    return risk2[0]
    #return np.std(x)

def sumrisk(weight,all_return):
    w = weight
    a = all_return
    portreturn = np.dot(np.array(all_return),np.array(weight))
    return risk(portreturn)

# 這是原本峻安要的輸出的內容，是以4個weight值為例
def bestweight_sample(w):
    return [
        risk(w[0]*all_return.iloc[:,0]) - np.mean(all_return.iloc[:,0])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,0]*w[0]) , 
        risk(w[1]*all_return.iloc[:,1]) - np.mean(all_return.iloc[:,1])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,1]*w[1]) , 
        risk(w[2]*all_return.iloc[:,2]) - np.mean(all_return.iloc[:,2])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,2]*w[2]) , 
        risk(w[3]*all_return.iloc[:,3]) - np.mean(all_return.iloc[:,3])*sumrisk(w,all_return) - np.mean(all_return.iloc[:,3]*w[3])
    ]

# 比較好的寫法，沒有限制weight的數量(亦即ETF的支數)
def bestweight(w):
    r = []
    restrict = sum(w) - 1
    r.append(restrict)
    for i in range(len(w)-1):
        v =(risk(w[i]*all_return.iloc[:,i]) \
            - (np.mean(all_return.iloc[:,i])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i]*w[i])-rfrate) )**2 \
            + (risk(w[i+1]*all_return.iloc[:,i+1]) \
            - (np.mean(all_return.iloc[:,i-1])-rfrate)*sumrisk(w,all_return) \
            - (np.mean(all_return.iloc[:,i+1]*w[i+1])-rfrate) )**2
        r.append(v) 
    return r

In [5]:
d1 = [-0.1,0.2,0.3,0.4]
d2 = [0.1,-0.2,-0.3,0.4]
d3 = [-0.1,0.3,0.4,0.5]
d4 = [-0.2,-0.4,0.5,0.6]
d5 = [-0.2,-0.4,0.6,0.8]
# d1-d5共有五個資產

rfrate = 0.2

# 這邊應該是要再多一個轉置，意即pd.DataFrame([d1,d2,d3,d4,d5]).T
all_return = pd.DataFrame([d1,d2,d3,d4,d5]).T
print(all_return.shape)

# weight應該有五位數
weight = [0.1,0.2,0.2,0.5,-0.1]

#r1 = fsolve(bestweight_sample, weight)
#print(r1)

r2 = fsolve(bestweight, weight)
print(r2)
all_return

(4, 5)


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


[ 1.69269368 -1.22439906  0.37661906  0.2436889  -0.10030928]


,0,1,2,3,4
0,-0.1,0.1,-0.1,-0.2,-0.2
1,0.2,-0.2,0.3,-0.4,-0.4
2,0.3,-0.3,0.4,0.5,0.6
3,0.4,0.4,0.5,0.6,0.8


In [6]:
d1 = [0.049,0.05,0.05,0.06]
d2 = [0.028,0.03,0.07,0.04]
all_return = pd.DataFrame([d1,d2]).T
print(all_return.shape)


#weight = [0.1,0.2,0.2,0.5]
weight = [0.9,0.1]
all_return
r2 = fsolve(bestweight, weight)
print(r2)

(4, 2)


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[0.88835515 0.13340924]


# 參考文章:
* [另外的python求解套件說明](https://github.com/wurmen/Gurobi-Python/blob/master/python-gurobi%20%20model/Python%2BGurobi%E5%BB%BA%E6%A8%A1.md)

In [7]:
#硬幹法  每個硬帶入 未考慮無風險資產
z_list = []     
x_list = []
d1 = [0.049,0.05,0.05,0.06]
d2 = [0.028,0.03,0.07,0.04]
all_return = pd.DataFrame([d1,d2]).T
for i in range(0,101,1):
    i = i/100
    z_list.append(sumrisk([i,1-i],all_return))
    x_list.append(i)
    
index = z_list.index(min(z_list))
x_list[index]

C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


0.71

# 比較sharpe ratio是否符合原本假設
## case1

In [8]:
import numpy as np
x = np.array([-0.5,0.5,0.9,-0.1,0.4,0.6])
y = np.array([1,-0.1,0.9,-0.2,0.2,-0.2])
rfrate = 0.2

In [9]:
from scipy import optimize
def f(a):
    return np.std(a*x+(1-a)*y)/(np.sum(a*x+(1-a)*y)-rfrate) #倒數最小，原本值最大
abc = optimize.fmin(f, 0.7)
abc[0]

Optimization terminated successfully.
         Current function value: 0.206043
         Iterations: 13
         Function evaluations: 26


0.5557617187499994

## case2

In [10]:
def risk(x):
    return np.var(x)

In [11]:
all_return = pd.DataFrame([x,y]).T
print(all_return.shape)

weight = [0.4,0.6]

r2 = fsolve(bestweight, weight)
print(r2)
all_return

(6, 2)
[0.56155456 0.43887423]


C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


,0,1
0,-0.5,1.0
1,0.5,-0.1
2,0.9,0.9
3,-0.1,-0.2
4,0.4,0.2
5,0.6,-0.2
